In [1]:
import pandas as pd 
import numpy as np 
from utils import * 
from src import get_genome_id
import os
from src.files import FASTAFile, InterProScanFile
import subprocess
from src.labeler import Labeler
from Bio.Align import PairwiseAligner

%load_ext autoreload
%autoreload 2

In [2]:
# I think the approach I will take is to be more strict about what I am considering real, and also handle pseudogenes separately. 

# To consider a sequence to be a true match, I will look at Prodigal sequences which are in-frame with non-pseudo reference sequences. 
# I don't want to require similar lengths (because this will miss the selenoproteins), but should perhaps check by comparing sequence
# identities. I will not consider any matches which are matches to suspect sequences. 

# For the pseudogenes, I will need to consider both in-frame and not in-frame hits. I should check, but I am assuming that every Prodigal-
# predicted sequence that is a translated component of a pseudogene should have query_overlap_fraction of 1. Also, will still need to require
# them being on the same strand. As a final check, I will run InterProScan

# I will run InterProScan on all intergenic sequences, and take those with AntiFam annotations to be spurious. I am hesitant to classify anything
# with no InterPro label as non-spurious, as I don't fully trust InterPro to capture everything. 

# Sequences that conflict with non-pseudo coding sequences and RNA genes are also potentially spurious. I will also run InterPro on the conflicts, and
# take anything which cannot be assigned an annotation, or is annotated as AntiFam. 

# Ultimately, going to want to run InterPro analysis on (1) potential pseudogene matches, (2) intergenic sequences, (3), conflicts. 

# I think I might want to write code to manually validate the pseudogene hits. 

# Maybe I should include the conflicting sequences with hypothetical sequences in the InterPro analysis?

In [ ]:
path = '../data/ref/GCF_013372265.1_summary.csv'
labeler = Labeler(path)
labeler.run()
labeler.to_df()

# assert ((df.top_hit_seq == 'none') & (~df.top_hit_pseudo) & (df.top_hit_feature == 'CDS')).sum() == 0, 'Expected all non-pseudo CDS genes to have a translated sequence.'
# assert ((df.top_hit_seq != 'none') & (df.top_hit_pseudo)).sum() == 0, 'Expected all pseudogenes to have no translated sequence.'


Labeler.__init__: 222 out of 2282 sequences have suspect top hits and are being removed.
Labeler._label_interpro: 2 out of 8 conflict sequences have no InterProScan hit.
Labeler._label_interpro: 6 out of 6 conflict sequences have an AntiFam InterProScan hit.
Labeler._label_interpro: 0 out of 16 match sequences have no InterProScan hit.
Labeler._label_interpro: 0 out of 16 match sequences have an AntiFam InterProScan hit.
Labeler._label_match: 1929 out of 2021 sequences have exact boundary matches.


get_alignment_scores: 100%|██████████| 92/92 [00:00<00:00, 396.93it/s]


Labeler._label_match: 92 out of 92 sequences meet the minimum alignment score threshold of 1.
Labeler._label_interpro: 14 out of 15 intergenic sequences have no InterProScan hit.
Labeler._label_interpro: 0 out of 1 intergenic sequences have an AntiFam InterProScan hit.
Labeler._label_auto: Automatically labeled 2027 sequences. 2060 could not be automatically-assigned labels.

NZ_CP054051.1_49 match
query_length: 357 top_hit_length: 348
overlap_length: 348 on the same strand
top_hit_product: preprotein translocase subunit SecG (pseudogene)
interpro: PANTHER PTHR34182 Preprotein translocase SecG subunit IPR004692

